In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
# import custom libraries
import sys
import os
import shutil
# sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
import hk_psql
import dash_bootstrap_components

import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go

# ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"

%load_ext autoreload
%autoreload 2

conn = hk_psql.connect_psql('mimic')


DATABASE CONNECTED !!!


In [2]:
def save_cohort(newdb, hadm_ids, subject_ids=None):
    list_queries=[]
    # hadm_ids=[20940957, 24181354]
    schemas = {
    # 'mimic_core':['transfers', 'admissions','patients'],
    'mimic_core':['transfers', 'admissions','patients'],

    'mimic_hosp':['labevents',
    'diagnoses_icd','procedures_icd','drgcodes',
    'emar'],
    'mimic_icu':['chartevents', 'inputevents', 'procedureevents', 'icustays'],
}
# newdb = 'co_1'

    for schema,tables in schemas.items():
        q=f"""
        DROP SCHEMA IF EXISTS {newdb}_{schema} CASCADE;
        CREATE SCHEMA IF NOT EXISTS {newdb}_{schema};
        """
        q=f"""
        CREATE SCHEMA IF NOT EXISTS {newdb}_{schema};
        """
        list_queries.append(q)
        # print(q) 
        # cmd(q,'mimic')
        for table in tables:
            print(schema, table)
            print(table)
            # copying table


            if table=='emar_detail':
                q=f"""
                    create table if not exists {newdb}_{schema}.{table}  AS
                    select emar_detail.emar_id, emar_detail.emar_seq, emar_detail.parent_field_ordinal, emar_detail.administration_type, emar_detail.pharmacy_id, emar_detail.barcode_type, emar_detail.reason_for_no_barcode, emar_detail.complete_dose_not_given, emar_detail.dose_due, emar_detail.dose_due_unit, emar_detail.dose_given, emar_detail.dose_given_unit, emar_detail.will_remainder_of_dose_be_given, emar_detail.product_amount_given, emar_detail.product_unit, emar_detail.product_code, emar_detail.product_description, emar_detail.product_description_other, emar_detail.prior_infusion_rate, emar_detail.infusion_rate, emar_detail.infusion_rate_adjustment, emar_detail.infusion_rate_adjustment_amount, emar_detail.infusion_rate_unit, emar_detail.route, emar_detail.infusion_complete, emar_detail.completion_interval, emar_detail.new_iv_bag_hung, emar_detail.continued_infusion_in_other_location, emar_detail.restart_interval, emar_detail.side, emar_detail.site, emar_detail.non_formulary_visual_verification
                    from {schema}.{table} emar_detail
                    inner join {newdb}_{schema}.emar emar
                    on emar.emar_id=emar_detail.emar_id

                    where hadm_id in ({str(hadm_ids)[1:-1]});
                    
                    
                    """
            elif table=='patients':
                q=f"""
                    create table if not exists {newdb}_{schema}.{table}  AS
                    select *
                    from {schema}.{table} 

                    where subject_id in ({str(subject_ids)[1:-1]});
                    
                    
                    """

            else:
                q=f"""
                create table if not exists {newdb}_{schema}.{table}  AS
                select * 
                from {schema}.{table}
                where 
                    hadm_id in ({str(hadm_ids)[1:-1]});
                """
            # print(q)
            list_queries.append(q)

            # cmd(q,'mimic')


    return list_queries

In [3]:
def save_cohort_tocsv(newdb,PATH):

    # PATH = f"datasets/{newdb}/"




    list_cmds=[]
    # hadm_ids=[20940957, 24181354]
    schemas = {
    'mimic_core':['transfers', 'admissions'],

    'mimic_hosp':[
        'labevents',
    'diagnoses_icd','procedures_icd','drgcodes',
    'emar',
    'd_hcpcs','d_icd_diagnoses','d_icd_procedures','d_labitems'],
    'mimic_icu':['chartevents', 'inputevents', 'procedureevents', 'icustays',
                 'd_items'],
}
# newdb = 'co_1'

    for schema,tables in schemas.items():

        

        for table in tables:
            print(schema, table)
            print(table)
            # copying table


        #     cmd=f"""
        #   \copy {newdb}_{schema}.{table} TO {PATH}\{schema[6:]}\{table}.csv WITH DELIMITER ',' CSV HEADER;


        # """
            cmd=f"""
          copy {newdb}{schema}.{table} TO '{PATH}{schema[6:]}/{table}.csv' WITH DELIMITER ',' CSV HEADER;


        """
            
            list_cmds.append(cmd)

            # cmd(q,'mimic')


    return list_cmds

# Select subset

# Export to CSV

In [77]:
newdb = 'sepsis_'
PATH = f"C:/DATA/Courses/DataVis/data-viz-project/MimicApp/datasets/{newdb[:-1]}/"

if os.path.exists(PATH+'core'):
    shutil.rmtree(PATH+'core')
if os.path.exists(PATH+'hosp'):
    shutil.rmtree(PATH+'hosp')
if os.path.exists(PATH+'icu'):
    shutil.rmtree(PATH+'icu')
os.makedirs(PATH+'core', exist_ok=True)
os.makedirs(PATH+'hosp', exist_ok=True)
os.makedirs(PATH+'icu', exist_ok=True)




# hadm_ids=[21038362,24941086]
# list_queries = save_cohort(newdb, hadm_ids, subject_ids=None)

list_cmds = save_cohort_tocsv(newdb,PATH)
list_cmds

text_file = open("copy_commands.txt", "w")
n = text_file.write("".join(list_cmds))
text_file.close()

mimic_core transfers
transfers
mimic_core admissions
admissions
mimic_hosp labevents
labevents
mimic_hosp diagnoses_icd
diagnoses_icd
mimic_hosp procedures_icd
procedures_icd
mimic_hosp drgcodes
drgcodes
mimic_hosp emar
emar
mimic_hosp d_hcpcs
d_hcpcs
mimic_hosp d_icd_diagnoses
d_icd_diagnoses
mimic_hosp d_icd_procedures
d_icd_procedures
mimic_hosp d_labitems
d_labitems
mimic_icu chartevents
chartevents
mimic_icu inputevents
inputevents
mimic_icu procedureevents
procedureevents
mimic_icu icustays
icustays
mimic_icu d_items
d_items


["\n          copy sepsis_mimic_core.transfers TO 'C:/DATA/Courses/DataVis/data-viz-project/MimicApp/datasets/sepsis/core/transfers.csv' WITH DELIMITER ',' CSV HEADER;\n\n\n        ",
 "\n          copy sepsis_mimic_core.admissions TO 'C:/DATA/Courses/DataVis/data-viz-project/MimicApp/datasets/sepsis/core/admissions.csv' WITH DELIMITER ',' CSV HEADER;\n\n\n        ",
 "\n          copy sepsis_mimic_hosp.labevents TO 'C:/DATA/Courses/DataVis/data-viz-project/MimicApp/datasets/sepsis/hosp/labevents.csv' WITH DELIMITER ',' CSV HEADER;\n\n\n        ",
 "\n          copy sepsis_mimic_hosp.diagnoses_icd TO 'C:/DATA/Courses/DataVis/data-viz-project/MimicApp/datasets/sepsis/hosp/diagnoses_icd.csv' WITH DELIMITER ',' CSV HEADER;\n\n\n        ",
 "\n          copy sepsis_mimic_hosp.procedures_icd TO 'C:/DATA/Courses/DataVis/data-viz-project/MimicApp/datasets/sepsis/hosp/procedures_icd.csv' WITH DELIMITER ',' CSV HEADER;\n\n\n        ",
 "\n          copy sepsis_mimic_hosp.drgcodes TO 'C:/DATA/Co

In [78]:
# connect to the existing database
conn = hk_psql.connect_psql('mimic', mypass='Hkmypassword1374!')
list_schema = hk_psql.get_schemas('mimic')



DATABASE CONNECTED !!!
c4_mimic_core.admissions
c4_mimic_core.transfers
c4_mimic_hosp.diagnoses_icd
c4_mimic_hosp.drgcodes
c4_mimic_hosp.emar
c4_mimic_hosp.labevents
c4_mimic_hosp.procedures_icd
c4_mimic_icu.chartevents
c4_mimic_icu.icustays
c4_mimic_icu.inputevents
c4_mimic_icu.procedureevents
cabg_mimic_core.admissions
cabg_mimic_core.transfers
cabg_mimic_hosp.diagnoses_icd
cabg_mimic_hosp.drgcodes
cabg_mimic_hosp.emar
cabg_mimic_hosp.labevents
cabg_mimic_hosp.procedures_icd
cabg_mimic_icu.chartevents
cabg_mimic_icu.icustays
cabg_mimic_icu.inputevents
cabg_mimic_icu.procedureevents
cohort3_mimic_core.admissions
cohort3_mimic_core.transfers
cohort3_mimic_hosp.diagnoses_icd
cohort3_mimic_hosp.drgcodes
cohort3_mimic_hosp.emar
cohort3_mimic_hosp.labevents
cohort3_mimic_hosp.procedures_icd
cohort3_mimic_icu.chartevents
cohort3_mimic_icu.icustays
cohort3_mimic_icu.inputevents
cohort3_mimic_icu.procedureevents
crp_mimic_core.admissions
crp_mimic_core.transfers
crp_mimic_hosp.diagnoses_icd
c

In [79]:
q = "".join(list_cmds)
hk_psql.cmd(q, 'mimic')

--- 47.78671360015869 seconds ---


# convert CSV to psql

In [4]:
# specify the name of new database.
# create this in advance using psql shell
newdb = 'datavis'

# connect to the existing database
conn = hk_psql.connect_psql(newdb, mypass='Hkmypassword1374!')
# list_schema = hk_psql.get_schemas(newdb)

DATABASE CONNECTED !!!


In [36]:
pre_schema = 'sepsis_'
PATH = f"C:/DATA/Courses/DataVis/data-viz-project/MimicApp/datasets/{pre_schema[:-1]}/"



schema_name = 'core'
q=f"""
        CREATE SCHEMA IF NOT EXISTS {pre_schema}{schema_name};
        """
q
hk_psql.cmd(q, newdb)

for csv_file in os.listdir(PATH+schema_name):
    table_name = csv_file[:-4] # remove .csv
    csv_file
    col_names = str(list(pd.read_csv(PATH+f'{schema_name}/'+csv_file).columns))[1:-1]
    col_names =  col_names.replace("'", "")
    col_names

    q=f"""
        DROP TABLE IF EXISTS {pre_schema}{schema_name}.{table_name};
        CREATE TABLE {pre_schema}{schema_name}.{table_name}({col_names});
        """
    q
    hk_psql.cmd(q, newdb)

    q=f"""
        COPY {pre_schema}{schema_name}.{table_name}({col_names})
        FROM '{PATH}{schema_name}/{csv_file}'
        DELIMITER ','
        CSV HEADER;
    
    """
    q
    hk_psql.cmd(q, newdb)
    term
    


'\n        CREATE SCHEMA IF NOT EXISTS sepsis_core;\n        '

--- 0.14374756813049316 seconds ---


'admissions.csv'

'subject_id, hadm_id, admittime, dischtime, deathtime, admission_type, admission_location, discharge_location, insurance, language, marital_status, ethnicity, edregtime, edouttime, hospital_expire_flag'

'\n        DROP TABLE IF EXISTS sepsis_core.admissions;\n        CREATE TABLE sepsis_core.admissions(subject_id, hadm_id, admittime, dischtime, deathtime, admission_type, admission_location, discharge_location, insurance, language, marital_status, ethnicity, edregtime, edouttime, hospital_expire_flag);\n        '

SyntaxError: syntax error at or near ";"
LINE 3: ...tus, ethnicity, edregtime, edouttime, hospital_expire_flag);
                                                                      ^


In [37]:
DBNAME='datavis'
hk_psql.get_schemas(DBNAME)

datavis_mimic_core.admissions
datavis_mimic_core.patients
datavis_mimic_core.transfers
datavis_mimic_hosp.d_hcpcs
datavis_mimic_hosp.d_icd_diagnoses
datavis_mimic_hosp.d_icd_procedures
datavis_mimic_hosp.d_labitems
datavis_mimic_hosp.diagnoses_icd
datavis_mimic_hosp.drgcodes
datavis_mimic_hosp.emar
datavis_mimic_hosp.labevents
datavis_mimic_hosp.procedures_icd
datavis_mimic_icu.chartevents
datavis_mimic_icu.d_items
datavis_mimic_icu.icustays
datavis_mimic_icu.inputevents
datavis_mimic_icu.procedureevents


['datavis_mimic_hosp', 'datavis_mimic_icu', 'datavis_mimic_core']